In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import GradScaler, autocast

from pathlib import Path
from PIL import Image
import numpy as np
import json
from tqdm import tqdm
import copy
from types import SimpleNamespace

# pycocotools가 필요합니다. pip install pycocotools
from pycocotools import mask as mask_utils

# --- 프로젝트 경로 추가 및 모듈 임포트 ---
import sys
sys.path.append('.')

from models.blade_model_v2 import BladeModelV2
from utils.criterion import SetCriterion
from utils.hungarian_matcher import HungarianMatcher

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

# --- 최종 설정 (Configuration) ---
class Config:
    DATA_ROOT = Path('C:/EngineBladeAI/EngineInspectionAI_MS/data/final_dataset_augmented')
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    # --- 학습 하이퍼파라미터 (수정) ---
    BATCH_SIZE = 4
    EPOCHS = 15 # <-- 1단계 학습을 위해 에포크 수 조정
    LR = 2e-5 
    WEIGHT_DECAY = 1e-4
    NUM_WORKERS = 0
    LR_DROP_STEP = 20
    GRADIENT_CLIP_VAL = 1.0 # <-- [추가] Gradient Clipping 값 설정

    MODEL = SimpleNamespace(
        # --- [수정] 딕셔너리를 SimpleNamespace로 변경 ---
        BACKBONE=SimpleNamespace(NAME='ConvNeXt-Tiny'),
        FPN=SimpleNamespace(OUT_CHANNELS=256),
        HEAD_B=SimpleNamespace(
            FEAT_CHANNELS=256,
            OUT_CHANNELS=256,
            NUM_CLASSES=3,
            QUERIES_PER_CLASS=100,
            DEC_LAYERS=6
        )
    )
    LOSS = SimpleNamespace(
        CLASS_WEIGHTS=[1.5, 1.0, 1.3], # Crack, Nick, Tear
        EOS_COEF=0.1
    )

config = Config()
print(f"\n--- Configuration Initialized ---")
print(f"Data Path: {config.DATA_ROOT}")
print(f"Device: {config.DEVICE}")
print(f"Initial Learning Rate: {config.LR}") # <-- 확인용 print 추가

PyTorch Version: 2.5.1+cu121
CUDA Available: True

--- Configuration Initialized ---
Data Path: C:\EngineBladeAI\EngineInspectionAI_MS\data\final_dataset_augmented
Device: cuda
Initial Learning Rate: 2e-05


In [2]:
class FinalBladeDataset(Dataset):
    """
    최종 통합된 데이터셋(final_dataset)을 위한 Dataset 클래스.
    """
    def __init__(self, root, split='train', transform=None):
        self.root = Path(root)
        self.split = split
        self.images_dir = self.root / self.split / 'images'
        
        json_path = self.root / self.split / 'annotations.json'
        with open(json_path, 'r') as f:
            self.data = json.load(f)
            
        self.images_info = self.data['images']
        self.annotations_map = {}
        for ann in self.data['annotations']:
            img_id = ann['image_id']
            if img_id not in self.annotations_map:
                self.annotations_map[img_id] = []
            self.annotations_map[img_id].append(ann)
            
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((640, 640)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transform

    def __len__(self):
        return len(self.images_info)

    def __getitem__(self, idx):
        img_info = self.images_info[idx]
        img_id = img_info['id']
        img_path = self.images_dir / img_info['file_name']
        image = Image.open(img_path).convert('RGB')
        
        original_w, original_h = image.size
        
        target = {}
        blade_mask = np.zeros((original_h, original_w), dtype=np.uint8)
        damage_masks_np = []
        damage_labels = []
        multilabel_vector = torch.zeros(3, dtype=torch.float32)

        annotations = self.annotations_map.get(img_id, [])
        for ann in annotations:
            # --- [핵심 수정] ---
            # segmentation 데이터가 유효한지 확인하는 방어 코드 추가
            seg = ann.get('segmentation')
            if not seg or not isinstance(seg, list) or not seg[0] or len(seg[0]) < 6:
                # 유효하지 않은 polygon (최소 3개의 점 필요)이면 건너뛰기
                continue
                
            cat_id = ann['category_id']
            
            try:
                rle = mask_utils.frPyObjects([seg[0]], original_h, original_w)
                mask = mask_utils.decode(rle)
            except Exception as e:
                print(f"Warning: Failed to decode segmentation for ann_id {ann.get('id')}. Error: {e}")
                continue

            if mask.ndim == 3:
                mask = np.max(mask, axis=2)

            if cat_id == 1:
                blade_mask = np.maximum(blade_mask, mask)
            else:
                damage_masks_np.append(mask)
                damage_labels.append(cat_id - 2)
                multilabel_vector[cat_id - 2] = 1.0

        image = self.transform(image)
        
        target['blade_mask'] = torch.from_numpy(blade_mask).long()
        target['labels'] = torch.tensor(damage_labels, dtype=torch.int64)
        target['multilabel'] = multilabel_vector
        
        if damage_masks_np:
            damage_masks_tensor = torch.from_numpy(np.stack(damage_masks_np)).float()
            target['masks'] = damage_masks_tensor
        else:
            target['masks'] = torch.zeros((0, original_h, original_w), dtype=torch.float32)

        return image, target

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    images = torch.stack(images, dim=0)
    return images, targets

print("✅ Dataset class and collate_fn are defined.")

✅ Dataset class and collate_fn are defined.


In [3]:
print("--- Creating DataLoaders ---")
train_dataset = FinalBladeDataset(root=config.DATA_ROOT, split='train')
val_dataset = FinalBladeDataset(root=config.DATA_ROOT, split='valid')

train_loader = DataLoader(
    train_dataset, batch_size=config.BATCH_SIZE, shuffle=True,
    num_workers=config.NUM_WORKERS, collate_fn=collate_fn
)
val_loader = DataLoader(
    val_dataset, batch_size=config.BATCH_SIZE, shuffle=False,
    num_workers=config.NUM_WORKERS, collate_fn=collate_fn
)
print(f"✅ DataLoaders created!")
print(f"   Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")

--- Creating DataLoaders ---
✅ DataLoaders created!
   Train samples: 7365, Val samples: 491


In [4]:
# ===== 셀 4: 모델, 손실함수, 옵티마이저 초기화 (수정) =====

print("--- Initializing Model, Criterion, Optimizer ---")
model = BladeModelV2(config).to(config.DEVICE)

matcher = HungarianMatcher(cost_class=2.0, cost_mask=5.0, cost_dice=5.0)
# 수정된 weight_dict (손상 탐지의 중요도를 크게 높임)
weight_dict = {'loss_ce': 5.0, 'loss_mask': 10.0, 'loss_dice': 10.0}

criterion = SetCriterion(
    num_classes=config.MODEL.HEAD_B.NUM_CLASSES, matcher=matcher, weight_dict=weight_dict,
    eos_coef=config.LOSS.EOS_COEF, losses=['labels', 'masks'],
    class_weights=config.LOSS.CLASS_WEIGHTS
).to(config.DEVICE)

optimizer = AdamW(model.parameters(), lr=config.LR, weight_decay=config.WEIGHT_DECAY)
scaler = GradScaler()

# --- [수정] 학습률 스케줄러 설정 ---
lr_scheduler = StepLR(optimizer, step_size=config.LR_DROP_STEP)

print("✅ Initialization complete.")

--- Initializing Model, Criterion, Optimizer ---
✅ Initialization complete.


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_42148\911795868.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [5]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

# torchmetrics 라이브러리 임포트
from torchmetrics.detection import MeanAveragePrecision
from torchmetrics.classification import MulticlassJaccardIndex

# ==============================================================================
# 학습 함수 (train_epoch)
# ==============================================================================
def train_epoch(model, dataloader, optimizer, device, epoch):
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{config.EPOCHS} [Stage 1: Train Blade]")
    
    for images, targets in pbar:
        images = images.to(device)
        targets_gpu = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        with autocast():
            outputs = model(images)
            
            # --- [1단계 수정] 블레이드 손실만 계산 ---
            blade_logits = outputs['blade_logits']
            gt_blade_masks = torch.stack([t['blade_mask'] for t in targets_gpu]).unsqueeze(1).float()
            blade_logits_resized = F.interpolate(blade_logits, size=gt_blade_masks.shape[-2:], mode="bilinear", align_corners=False)
            
            loss_blade = F.binary_cross_entropy_with_logits(blade_logits_resized, gt_blade_masks)
            weighted_loss = loss_blade # 최종 손실은 이제 블레이드 손실만 해당

        optimizer.zero_grad()
        scaler.scale(weighted_loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.GRADIENT_CLIP_VAL)
        scaler.step(optimizer)
        scaler.update()

        total_loss += weighted_loss.item()
        pbar.set_postfix({'loss': f'{weighted_loss.item():.4f}'})
        
    return total_loss / len(dataloader)

def validate(model, dataloader, device):
    model.eval()
    
    blade_iou_metric = MulticlassJaccardIndex(num_classes=2).to(device)
    val_losses = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="[Valid]")
        for images, targets in pbar:
            images = images.to(device)
            targets_gpu = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            with autocast():
                outputs = model(images)
                
                # --- [1단계 수정] 블레이드 손실만 계산 ---
                blade_logits = outputs['blade_logits']
                gt_blade_masks = torch.stack([t['blade_mask'] for t in targets_gpu]).unsqueeze(1).float()
                blade_logits_resized = F.interpolate(blade_logits, size=gt_blade_masks.shape[-2:], mode="bilinear", align_corners=False)
                loss_blade = F.binary_cross_entropy_with_logits(blade_logits_resized, gt_blade_masks)
            
            val_losses.append(loss_blade.item())
            
            # Blade IoU 계산
            pred_blade_masks = (torch.sigmoid(blade_logits_resized) > 0.5).int().squeeze(1)
            blade_iou_metric.update(pred_blade_masks, gt_blade_masks.squeeze(1).int())

    metrics = {
        'loss': np.mean(val_losses),
        'blade_iou': blade_iou_metric.compute().item(),
    }
    
    return metrics

In [ ]:
# ===== 셀 6의 메인 학습 루프를 아래 코드로 교체 =====

print("\n--- 🚀 Starting Stage 1 Training: Blade Expert 🚀 ---")
best_val_loss = float('inf')

for epoch in range(config.EPOCHS):
    # --- [수정] criterion 인자 제거 ---
    train_loss = train_epoch(model, train_loader, optimizer, config.DEVICE, epoch)
    val_metrics = validate(model, val_loader, config.DEVICE)
    
    val_loss = val_metrics['loss']
    
    print(f"\nEpoch {epoch+1}/{config.EPOCHS} -> Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    print(f"  [Metrics] Blade IoU: {val_metrics['blade_iou']:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'blade_expert_model.pth')
        print(f"✨ New best blade expert model saved with validation loss: {best_val_loss:.4f}")

print("\n--- 🎉 Stage 1 Training Complete ---")


--- 🚀 Starting Stage 1 Training: Blade Expert 🚀 ---


Epoch 1/15 [Stage 1: Train Blade]:   0%|          | 0/1842 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_42148\3467118361.py:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[Valid]:   0%|          | 0/123 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_42148\3467118361.py:57: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[Valid]: 100%|██████████| 123/123 [00:09<00:00, 12.74it/s]



Epoch 1/15 -> Train Loss: 0.2651, Val Loss: 0.0800
  [Metrics] Blade IoU: 0.9418
✨ New best blade expert model saved with validation loss: 0.0800


[Valid]: 100%|██████████| 123/123 [00:09<00:00, 12.76it/s]



Epoch 2/15 -> Train Loss: 0.1812, Val Loss: 0.0729
  [Metrics] Blade IoU: 0.9498
✨ New best blade expert model saved with validation loss: 0.0729


[Valid]: 100%|██████████| 123/123 [00:09<00:00, 12.77it/s]



Epoch 3/15 -> Train Loss: 0.1533, Val Loss: 0.0726
  [Metrics] Blade IoU: 0.9502
✨ New best blade expert model saved with validation loss: 0.0726


Epoch 4/15 [Stage 1: Train Blade]:  39%|███▉      | 722/1842 [01:39<02:35,  7.18it/s, loss=0.8343]

In [ ]:
print("\n--- 🚀 Initializing Stage 2: Damage Expert Training 🚀 ---")

# 1. 새로운 모델 객체 생성
# (이전 학습의 영향을 받지 않는 깨끗한 모델로 시작)
model = BladeModelV2(config).to(config.DEVICE)

# 2. 1단계에서 학습한 '블레이드 전문가' 가중치 불러오기
# strict=False는 Head-B의 가중치가 없어도 오류를 내지 않도록 함
model.load_state_dict(torch.load('blade_expert_model.pth'), strict=False)
print("✅ Blade expert weights loaded successfully.")

# 3. Backbone과 Head-A의 가중치 동결 (Freeze)
for name, param in model.named_parameters():
    if 'head_b' not in name:
        param.requires_grad = False
        
print("✅ Backbone and Head-A have been frozen.")

# 4. 학습 가능한 파라미터만 필터링하여 새로운 옵티마이저 생성
# (오직 Head-B의 가중치만 업데이트됨)
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = AdamW(trainable_params, lr=config.LR, weight_decay=config.WEIGHT_DECAY)
lr_scheduler = StepLR(optimizer, step_size=config.LR_DROP_STEP)

print("✅ Optimizer re-initialized for Head-B only.")

# 5. 손상 탐지용 Criterion 재정의 (셀 4와 동일)
matcher = HungarianMatcher(cost_class=2.0, cost_mask=5.0, cost_dice=5.0)
weight_dict = {'loss_ce': 2.0, 'loss_mask': 5.0, 'loss_dice': 5.0}
criterion = SetCriterion(
    num_classes=config.MODEL.HEAD_B.NUM_CLASSES, matcher=matcher, weight_dict=weight_dict,
    eos_coef=config.LOSS.EOS_COEF, losses=['labels', 'masks'],
    class_weights=config.LOSS.CLASS_WEIGHTS
).to(config.DEVICE)

In [ ]:
def train_epoch_stage2(model, criterion, dataloader, optimizer, device, epoch):
    model.train()
    # Head-B만 학습 모드로 설정 (BatchNorm 등 레이어에 영향)
    model.head_b.train()
    
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{config.EPOCHS} [Stage 2: Train Damage]")
    
    for images, targets in pbar:
        images = images.to(device)
        targets_gpu = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        with autocast():
            outputs = model(images)
            
            # --- [2단계] 손상 탐지(Head-B) 손실만 계산 ---
            loss_dict = criterion(outputs, targets_gpu)
            weighted_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        optimizer.zero_grad()
        scaler.scale(weighted_loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.GRADIENT_CLIP_VAL)
        scaler.step(optimizer)
        scaler.update()

        total_loss += weighted_loss.item()
        pbar.set_postfix({'loss': f'{weighted_loss.item():.4f}'})
        
    return total_loss / len(dataloader)

# ==============================================================================
# 검증 함수 (validate)
# ==============================================================================
def validate(model, criterion, dataloader, device):
    model.eval()
    criterion.eval()
    
    # 평가 지표 객체 초기화
    blade_iou_metric = MulticlassJaccardIndex(num_classes=2).to(device)
    map_metric = MeanAveragePrecision(iou_type="segm") # 경고 메시지 제거

    val_losses = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="[Valid]")
        for images, targets in pbar:
            images = images.to(device)
            targets_gpu = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            with autocast():
                outputs = model(images)
                # --- 손실 계산 ---
                loss_dict = criterion(outputs, targets_gpu)
                damage_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
                blade_logits = outputs['blade_logits']
                gt_blade_masks = torch.stack([t['blade_mask'] for t in targets_gpu]).unsqueeze(1).float()
                blade_logits_resized = F.interpolate(blade_logits, size=gt_blade_masks.shape[-2:], mode="bilinear", align_corners=False)
                loss_blade = F.binary_cross_entropy_with_logits(blade_logits_resized, gt_blade_masks)
                weighted_loss = damage_loss + (loss_blade * 1.0)
            val_losses.append(weighted_loss.item())

            # --- 평가 지표 업데이트 ---
            # 1. Blade IoU
            pred_blade_masks = (torch.sigmoid(blade_logits_resized) > 0.5).int().squeeze(1)
            blade_iou_metric.update(pred_blade_masks, gt_blade_masks.squeeze(1).int())

            # 2. mAP 계산을 위한 데이터 형식 변환
            # (이전의 가장 안정적인 버전으로 복구)
            pred_logits_cpu = outputs['pred_logits'].cpu()
            pred_masks_cpu = outputs['pred_masks'].cpu()
            
            preds_for_map = []
            for i in range(len(targets)):
                scores, labels = F.softmax(pred_logits_cpu[i], dim=-1).max(-1)
                masks_bool = (torch.sigmoid(pred_masks_cpu[i]) > 0.5)
                preds_for_map.append(dict(masks=masks_bool, scores=scores, labels=labels))

            targets_for_map = []
            for t in targets:
                targets_for_map.append(dict(masks=(t['masks'] > 0.5), labels=t['labels']))
            
            map_metric.update(preds_for_map, targets_for_map)

    # 최종 결과 집계
    blade_iou = blade_iou_metric.compute().item()
    map_results = map_metric.compute()
    
    # mAP 결과에서 Precision, Recall, F1-score 근사치 계산
    precision = map_results['map_50'].item()
    recall = map_results['mar_100'].item()
    f1 = 2 * (precision * recall) / (precision + recall + 1e-6)

    metrics = {
        'loss': np.mean(val_losses),
        'blade_iou': blade_iou,
        'mAP': map_results['map'].item(),
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }
    
    return metrics

# validate 함수는 이전의 최종 버전을 그대로 사용해도 됩니다.
# (모든 지표를 계산하는 최종 validate 함수)

In [ ]:
print("\n--- 🚀 Starting Stage 2 Training: Damage Expert 🚀 ---")
best_val_loss = float('inf')

# 2단계 학습을 위해 EPOCHS를 다시 설정할 수 있습니다.
# 예: config.EPOCHS = 30

for epoch in range(config.EPOCHS):
    train_loss = train_epoch_stage2(model, criterion, train_loader, optimizer, config.DEVICE, epoch)
    val_metrics = validate(model, criterion, val_loader, config.DEVICE)
    
    val_loss = val_metrics['loss']
    
    print(f"\nEpoch {epoch+1}/{config.EPOCHS} -> Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    print(f"  [Damage] mAP: {val_metrics['mAP']:.4f} | Precision: {val_metrics['precision']:.4f} | "
          f"Recall: {val_metrics['recall']:.4f} | F1: {val_metrics['f1_score']:.4f}")
    
    lr_scheduler.step()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'final_best_model.pth') # <-- 최종 모델 저장
        print(f"✨ New best FINAL model saved with validation loss: {best_val_loss:.4f}")

print("\n--- 🎉 Stage 2 Training Complete ---")

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

# ===== visualize_predictions 함수 교체 =====

def visualize_predictions(model, dataloader, device, num_samples=3, conf_threshold=0.5):
    """모델의 예측 결과를 시각화합니다."""
    model.eval()
    samples_shown = 0
    
    # Config에 클래스 이름 리스트가 있다고 가정
    # 예: config.MODEL.HEAD_B.CLASSES = ['crack', 'nick', 'tear']
    class_names = getattr(config.MODEL.HEAD_B, 'CLASSES', ['crack', 'nick', 'tear'])
    
    with torch.no_grad():
        for images, targets in dataloader:
            if samples_shown >= num_samples:
                break
                
            images = images.to(device)
            outputs = model(images)
            
            images_cpu = images.cpu()
            outputs_cpu = {k: v.cpu() for k, v in outputs.items() if torch.is_tensor(v)}
            
            for i in range(images.size(0)):
                if samples_shown >= num_samples:
                    break
                
                image_tensor = images_cpu[i]
                target = targets[i]
                output = {k: v[i] for k, v in outputs_cpu.items()}
                
                # 이미지를 Numpy 배열로 변환
                img_to_draw = image_tensor.permute(1, 2, 0).numpy()
                mean = np.array([0.485, 0.456, 0.406])
                std = np.array([0.229, 0.224, 0.225])
                img_to_draw = (img_to_draw * std + mean) * 255
                img_to_draw = np.clip(img_to_draw, 0, 255).astype(np.uint8)
                
                # --- [핵심 수정] 메모리 레이아웃을 OpenCV 호환 형태로 변경 ---
                img_to_draw = np.ascontiguousarray(img_to_draw)
                
                plt.figure(figsize=(12, 12))
                
                # 1. 정답 마스크 그리기 (초록색, 점선)
                for mask in target['masks']:
                    contours, _ = cv2.findContours(mask.numpy().astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    cv2.drawContours(img_to_draw, contours, -1, (0, 255, 0), 2)

                # 2. 예측 마스크 그리기 (빨간색, 실선)
                scores, labels = F.softmax(output['pred_logits'], dim=-1).max(-1)
                
                for j, (score, label) in enumerate(zip(scores, labels)):
                    if score > conf_threshold:
                        mask = (torch.sigmoid(output['pred_masks'][j]) > 0.5).numpy().astype(np.uint8)
                        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        cv2.drawContours(img_to_draw, contours, -1, (255, 0, 0), 2)
                        
                        cat_name = class_names[label.item()]
                        text = f'{cat_name}: {score:.2f}'
                        if contours:
                            x, y, w, h = cv2.boundingRect(contours[0])
                            cv2.putText(img_to_draw, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                plt.imshow(img_to_draw)
                plt.title(f"Sample {samples_shown+1} (Green: Ground Truth, Red: Prediction)")
                plt.axis('off')
                plt.show()
                
                samples_shown += 1

In [ ]:
# 모델 불러오기
model.load_state_dict(torch.load('blade_damage_best_model.pth'))
# 시각화 함수 실행
visualize_predictions(model, val_loader, config.DEVICE)